In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = '/Users/mymac/Google Drive/My Drive/Forex_Robot/'

In [3]:
currency_pair = 'Eur_Usd'
years = '2012-2022'

In [4]:
df = pd.read_csv(file_path + f'Oanda_{currency_pair}_M15_{years}.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

df_long = pd.read_csv(file_path + f'Oanda_{currency_pair}_H1_{years}.csv')
df_long.Date = pd.to_datetime(df_long.Date)
df_long.reset_index(drop=True, inplace=True)

In [5]:
def add_fractal(df, i, look_back=3):
    if i >= look_back and i < df.shape[0] - look_back:
        lows = []
        highs = []

        for j in range(1, look_back + 1):
            prev_mid_low, prev_mid_high = df.loc[df.index[i - j], ['Mid_Low', 'Mid_High']]
            future_mid_low, future_mid_high = df.loc[df.index[i + j], ['Mid_Low', 'Mid_High']]

            lows.append(float(prev_mid_low))
            lows.append(float(future_mid_low))
            highs.append(float(prev_mid_high))
            highs.append(float(future_mid_high))

        mid_low, mid_high = df.loc[df.index[i], ['Mid_Low', 'Mid_High']]

        if float(mid_low) < min(lows):
            return float(mid_low), 1.0

        elif float(mid_high) > max(highs):
            return float(mid_high), 0.0

        else:
            return np.nan, np.nan

    else:
        return np.nan, np.nan

def psar(barsdata, iaf=0.02, maxaf=0.2):
    length = len(barsdata)
    high = list(barsdata['Mid_High'])
    low = list(barsdata['Mid_Low'])
    close = list(barsdata['Mid_Close'])
    psar = close[0:len(close)]
    bull = True
    af = iaf
    hp = high[0]
    lp = low[0]
    for i in range(2, length):
        if bull:
            psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
        else:
            psar[i] = psar[i - 1] + af * (lp - psar[i - 1])
        reverse = False
        if bull:
            if low[i] < psar[i]:
                bull = False
                reverse = True
                psar[i] = hp
                lp = low[i]
                af = iaf
        else:
            if high[i] > psar[i]:
                bull = True
                reverse = True
                psar[i] = lp
                hp = high[i]
                af = iaf
        if not reverse:
            if bull:
                if high[i] > hp:
                    hp = high[i]
                    af = min(af + iaf, maxaf)
                if low[i - 1] < psar[i]:
                    psar[i] = low[i - 1]
                if low[i - 2] < psar[i]:
                    psar[i] = low[i - 2]
            else:
                if low[i] < lp:
                    lp = low[i]
                    af = min(af + iaf, maxaf)
                if high[i - 1] > psar[i]:
                    psar[i] = high[i - 1]
                if high[i - 2] > psar[i]:
                    psar[i] = high[i - 2]
    return psar


def atr(high, low, close, lookback=14):
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).sum() / lookback


def rsi(closes, periods=14):
    close_delta = closes.diff()

    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))

    return rsi

  
def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()

    return adx_smooth


def stoch(high, low, close, lookback=14):
    high_lookback = high.rolling(lookback).max()
    low_lookback = low.rolling(lookback).min()
    slow_k = (close - low_lookback) * 100 / (high_lookback - low_lookback)
    slow_d = slow_k.rolling(3).mean()

    return slow_k, slow_d

def stoch_rsi(data, k_window=3, d_window=3, window=14):
    min_val = data.rolling(window=window, center=False).min()
    max_val = data.rolling(window=window, center=False).max()

    stoch = ((data - min_val) / (max_val - min_val)) * 100

    slow_k = stoch.rolling(window=k_window, center=False).mean()

    slow_d = slow_k.rolling(window=d_window, center=False).mean()

    return slow_k, slow_d

def n_macd(macd, macdsignal, lookback=50):
    n_macd = 2 * (((macd - macd.rolling(lookback).min()) / (macd.rolling(lookback).max() - macd.rolling(lookback).min()))) - 1
    n_macdsignal = 2 * (((macdsignal - macdsignal.rolling(lookback).min()) / (macdsignal.rolling(lookback).max() - macdsignal.rolling(lookback).min()))) - 1

    return n_macd, n_macdsignal

def chop(df, lookback=14):
    atr1 = atr(df, lookback=1)
    high, low = df['Mid_High'], df['Mid_Low']

    chop = np.log10(atr1.rolling(lookback).sum() / (high.rolling(lookback).max() - low.rolling(lookback).min())) / np.log10(lookback)

    return chop

def vo(volume, short_lookback=5, long_lookback=10):
    short_ema =  pd.Series.ewm(volume, span=short_lookback).mean()
    long_ema = pd.Series.ewm(volume, span=long_lookback).mean()

    volume_oscillator = (short_ema - long_ema) / long_ema

    return volume_oscillator

def bar_lengths(bar_lens, window=36):
    return bar_lens.rolling(window=window).mean(), bar_lens.rolling(window=window).std()

def sar_lengths(opens, sars, window=36):
    diffs = abs(opens - sars.shift(1))

    return diffs.rolling(window=window).mean(), diffs.rolling(window=window).std()

def supertrend(barsdata, atr_len=3, mult=3):
    curr_atr = atr(barsdata['Mid_High'], barsdata['Mid_Low'], barsdata['Mid_Close'], lookback=atr_len)
    highs, lows = barsdata['Mid_High'], barsdata['Mid_Low']
    hl2 = (highs + lows) / 2
    final_upperband = upper_band = hl2 + mult * curr_atr
    final_lowerband = lower_band = hl2 - mult * curr_atr

    # initialize Supertrend column to True
    supertrend = [True] * len(df)

    close = barsdata['Mid_Close']
    
    for i in range(1, len(df.index)):
        curr, prev = i, i - 1
        
        # if current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True

        # if current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False

        # else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]

            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

    return supertrend, final_upperband, final_lowerband

# def supertrend(barsdata, atr_len=3, mult=3):
#     curr_atr = atr(barsdata['ha_high'], barsdata['ha_low'], barsdata['ha_close'], lookback=atr_len)
#     highs, lows = barsdata['ha_high'], barsdata['ha_low']
#     hl2 = (highs + lows) / 2
#     final_upperband = upper_band = hl2 + mult * curr_atr
#     final_lowerband = lower_band = hl2 - mult * curr_atr

#     # initialize Supertrend column to True
#     supertrend = [True] * len(df)

#     close = barsdata['ha_close']
    
#     for i in range(1, len(df.index)):
#         curr, prev = i, i - 1
        
#         # if current close price crosses above upperband
#         if close[curr] > final_upperband[prev]:
#             supertrend[curr] = True

#         # if current close price crosses below lowerband
#         elif close[curr] < final_lowerband[prev]:
#             supertrend[curr] = False

#         # else, the trend continues
#         else:
#             supertrend[curr] = supertrend[prev]
            
#             # adjustment to the final bands
#             if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
#                 final_lowerband[curr] = final_lowerband[prev]

#             if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
#                 final_upperband[curr] = final_upperband[prev]

#     return supertrend, final_upperband, final_lowerband

def heikin_ashi(opens, highs, lows, closes):
    ha_close = list((opens + highs + lows + closes) / 4)
    ha_opens = []

    opens_list, closes_list = list(opens), list(closes)

    for i in range(len(ha_close)):
        if i == 0:
            ha_opens.append((opens_list[i] + closes_list[i]) / 2)

        else:
            ha_opens.append((ha_opens[i - 1] + ha_close[i - 1]) / 2)

    ha_highs = list(pd.DataFrame({'ha_open': ha_opens, 'ha_close': ha_close, 'high': list(highs)}).max(axis=1))
    ha_lows = list(pd.DataFrame({'ha_open': ha_opens, 'ha_close': ha_close, 'low': list(lows)}).min(axis=1))

    return ha_opens, ha_highs, ha_lows, ha_close

def trend_indicator(opens, highs, lows, closes, ema_period=50, smoothing_period=10):
    ha_open, ha_high, ha_low, ha_close = heikin_ashi(opens, highs, lows, closes)

    ha_o_ema = pd.Series.ewm(pd.DataFrame({'ha_open': ha_open}), span=ema_period).mean()
    ha_h_ema = pd.Series.ewm(pd.DataFrame({'ha_high': ha_high}), span=ema_period).mean()
    ha_l_ema = pd.Series.ewm(pd.DataFrame({'ha_low': ha_low}), span=ema_period).mean()
    ha_c_ema = pd.Series.ewm(pd.DataFrame({'ha_close': ha_close}), span=ema_period).mean()

    return pd.Series.ewm(ha_o_ema, span=smoothing_period).mean(), pd.Series.ewm(ha_h_ema, span=smoothing_period).mean(), pd.Series.ewm(ha_l_ema, span=smoothing_period).mean(), pd.Series.ewm(ha_c_ema, span=smoothing_period).mean()

def qqe_mod(barsdata, rsi_period=6, smoothing=5, qqe_factor=5, qqe2_factor=1.61, threshold=3, mult=0.35, sma_length=50):
    wilders_period = rsi_period * 2 - 1

    curr_rsi = rsi(barsdata, periods=rsi_period)
    rsi_ema = pd.Series.ewm(curr_rsi, span=smoothing).mean()
    atr_rsi = abs(rsi_ema.shift(1) - rsi_ema)
    atr_rsi_ema = pd.Series.ewm(atr_rsi, span=wilders_period).mean()
    dar = pd.Series.ewm(atr_rsi_ema, span=wilders_period).mean() * qqe_factor

    newshortband = rsi_ema + dar
    newlongband = rsi_ema - dar

    rsi_ema_list = list(rsi_ema)

    longband = [0]
    for i in range(1, len(rsi_ema_list)):
        if rsi_ema_list[i - 1] > longband[i - 1] and rsi_ema_list[i] > longband[i - 1]:
            longband.append(max(longband[i - 1],newlongband[i]))

        else:
            longband.append(newlongband[i])

    shortband = [0]
    for i in range(1,len(rsi_ema_list)):
        if rsi_ema_list[i - 1] < shortband[i - 1] and rsi_ema_list[i] < shortband[i - 1]:
            shortband.append(min(shortband[i - 1],newshortband[i]))
            
        else:
            shortband.append(newshortband[i])

    longband = pd.Series(longband)
    shortband = pd.Series(shortband)

    trend = np.where(rsi_ema > longband.shift(1), 1, -1)    
    fastatrrsitl = pd.Series(np.where(trend == 1, longband, shortband))

    basis = (fastatrrsitl - 50).rolling(window=sma_length).mean()
    dev = (fastatrrsitl - 50).rolling(window=sma_length).std() * mult
    upper = basis + dev
    lower = basis - dev

    greenbar1 = rsi_ema - 50 > threshold
    greenbar2 = rsi_ema - 50 > upper
    redbar1 = rsi_ema - 50 < threshold
    redbar2 = rsi_ema - 50 < lower

    # uptrend = np.where((greenbar1 & greenbar2), True, False)
    # downtrend = np.where((redbar1 & redbar2), True, False)

    uptrend = np.where((greenbar2), True, False)
    downtrend = np.where((redbar2), True, False)

    return uptrend, downtrend

def williams_r(highs, lows, closes, length=21, ema_length=15):
    highest_highs = highs.rolling(window=length).max()
    lowest_lows = lows.rolling(window=length).min()

    willy = 100 * (closes - highest_highs) / (highest_highs - lowest_lows)
    willy_ema = pd.Series.ewm(willy, span=ema_length).mean()

    return willy, willy_ema

def rh_irb(opens, highs, lows, closes, z=45):
    a = abs(highs - lows)
    b = abs(closes - opens)
    c = z / 100

    rv = b < c * a

    x = lows + (c * a)
    y = highs - (c * a)

    sl = (rv) & (highs > y) & (closes < y) & (opens < y)
    ss = (rv) & (lows < x) & (closes > x) & (opens > x)

    return sl, ss

In [6]:
df['ema200'] = pd.Series.ewm(df['Mid_Close'], span=200).mean()
df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['sl'], df['ss'] = rh_irb(df['Mid_Open'], df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df_long['ema200'] = pd.Series.ewm(df_long['Mid_Close'], span=200).mean()

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [7]:
df = pd.merge(df, df_long, how='left', on='Date')
df.reset_index(drop=True, inplace=True)
df = df.fillna(method='ffill')
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [8]:
df.head()

,Date,Bid_Open_x,Bid_High_x,Bid_Low_x,Bid_Close_x,Ask_Open_x,Ask_High_x,Ask_Low_x,Ask_Close_x,Mid_Open_x,...,Ask_Open_y,Ask_High_y,Ask_Low_y,Ask_Close_y,Mid_Open_y,Mid_High_y,Mid_Low_y,Mid_Close_y,Volume_y,ema200_y
0,2012-01-01 22:00:00,1.29359,1.29407,1.29359,1.29368,1.29459,1.29507,1.29459,1.29468,1.29409,...,1.29459,1.29520,1.29432,1.29508,1.29409,1.29470,1.29382,1.29458,102.0,1.293916
1,2012-01-01 22:15:00,1.29372,1.29389,1.29372,1.29389,1.29472,1.29489,1.29472,1.29489,1.29422,...,1.29459,1.29520,1.29432,1.29508,1.29409,1.29470,1.29382,1.29458,102.0,1.293916
2,2012-01-01 22:30:00,1.29382,1.29420,1.29332,1.29404,1.29482,1.29520,1.29432,1.29504,1.29432,...,1.29459,1.29520,1.29432,1.29508,1.29409,1.29470,1.29382,1.29458,102.0,1.293916
3,2012-01-01 22:45:00,1.29409,1.29413,1.29393,1.29408,1.29509,1.29513,1.29493,1.29508,1.29459,...,1.29459,1.29520,1.29432,1.29508,1.29409,1.29470,1.29382,1.29458,102.0,1.293916
4,2012-01-01 23:00:00,1.29403,1.29464,1.29403,1.29449,1.29503,1.29524,1.29487,1.29499,1.29453,...,1.29503,1.29559,1.29387,1.29517,1.29453,1.29534,1.29362,1.29467,233.0,1.294027


In [9]:
df.tail()

,Date,Bid_Open_x,Bid_High_x,Bid_Low_x,Bid_Close_x,Ask_Open_x,Ask_High_x,Ask_Low_x,Ask_Close_x,Mid_Open_x,...,Ask_Open_y,Ask_High_y,Ask_Low_y,Ask_Close_y,Mid_Open_y,Mid_High_y,Mid_Low_y,Mid_Close_y,Volume_y,ema200_y
249193,2021-12-31 20:45:00,1.13839,1.13841,1.13749,1.13749,1.13856,1.13858,1.13767,1.13767,1.13848,...,1.13832,1.13863,1.13767,1.13767,1.13826,1.13854,1.13758,1.13758,623.0,1.131949
249194,2021-12-31 21:00:00,1.13751,1.13783,1.13751,1.13781,1.13768,1.13801,1.13768,1.13799,1.13760,...,1.13768,1.13838,1.13745,1.13780,1.13760,1.13827,1.13727,1.13730,740.0,1.132002
249195,2021-12-31 21:15:00,1.13780,1.13787,1.13756,1.13782,1.13797,1.13806,1.13771,1.13798,1.13788,...,1.13768,1.13838,1.13745,1.13780,1.13760,1.13827,1.13727,1.13730,740.0,1.132002
249196,2021-12-31 21:30:00,1.13782,1.13785,1.13771,1.13774,1.13798,1.13802,1.13787,1.13791,1.13790,...,1.13768,1.13838,1.13745,1.13780,1.13760,1.13827,1.13727,1.13730,740.0,1.132002
249197,2021-12-31 21:45:00,1.13771,1.13816,1.13677,1.13680,1.13790,1.13838,1.13745,1.13780,1.13780,...,1.13768,1.13838,1.13745,1.13780,1.13760,1.13827,1.13727,1.13730,740.0,1.132002


In [10]:
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]

In [11]:
rounding = 5 if 'Jpy' not in currency_pair else 3
divider = 10000 if 'Jpy' not in currency_pair else 100
n_recent_trades = 2
recent_hours = 1

In [12]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def run_simulation(risk_reward_ratio, spread_cutoff, cushion):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    trade = None
    cushion /= divider

    for i in range(1, len(df)):
        curr_bid_open, curr_bid_high, curr_bid_low, curr_ask_open, curr_ask_high, curr_ask_low, curr_mid_open, curr_date = df.loc[df.index[i], ['Bid_Open_x', 'Bid_High_x', 'Bid_Low_x', 'Ask_Open_x', 'Ask_High_x', 'Ask_Low_x', 'Mid_Open_x', 'Date']]
        ema, ema_long, sl, ss, prev_high, prev_low, adx = df.loc[df.index[i - 1], ['ema200_x', 'ema200_y', 'sl', 'ss', 'Mid_High_x', 'Mid_Low_x', 'adx']]
        spread = abs(curr_ask_open - curr_bid_open)

        buy_signal = ema > ema_long and sl and curr_ask_high >= prev_high and adx > 25
        sell_signal = ema < ema_long and ss and curr_bid_low <= prev_low and adx > 25

        if trade is None:
            if buy_signal:
                open_price = float(prev_high)

                stop_loss = prev_low - cushion
                stop_loss = round(stop_loss, rounding)

                if stop_loss < open_price:
                    curr_pips_to_risk = open_price - stop_loss

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price + (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units('buy', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                        'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                                        'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_buys += 1

                        pips_risked.append(curr_pips_to_risk)

            elif sell_signal:
                open_price = float(prev_low)
                
                stop_loss = prev_high + cushion
                stop_loss = round(stop_loss, rounding)

                if stop_loss > open_price:
                    curr_pips_to_risk = stop_loss - open_price

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price - (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units('sell', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_sells += 1

                        pips_risked.append(curr_pips_to_risk)

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            # trade_amount = -50.0
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None    

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            # trade_amount = 50.0 * risk_reward_ratio
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            # trade_amount = -50.0
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            # trade_amount = 50.0 * risk_reward_ratio
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [13]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1.0, 1.5, 2.0]
spread_cutoffs = [0.10, 0.20]
cushions = [2, 3, 4, 5]

n_possibilities = len(risk_reward_ratio_vals) * len(spread_cutoffs) * len(cushions)
all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
    for spread_val in spread_cutoffs:
        for cushion in cushions:
            all_combos.append((risk_reward_ratio, spread_val, cushion))

percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_cushion = None
top_n_results = 10
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, cushion in combos_to_try:
    reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_simulation(risk_reward_ratio, spread_val, cushion)
    runs_finished += 1

    print(reward)
    print('Num buys: ' + str(n_sells))
    print('Num sells: ' + str(n_buys))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    if len(pips_risked) > 0:
        print('Avg pips risked: ' + str(np.array(pips_risked).mean()))
    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or reward > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread': spread_val, 'cushion': cushion})


    if reward > best_reward:
        best_reward = reward
        best_risk_reward = risk_reward_ratio
        best_spread_cutoff = spread_val
        best_cushion = cushion

    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 24

-23952.752730000368
Num buys: 707
Num sells: 767
Num trades: 1474
Num wins: 408
Num losses: 1066
Win streak: 6
Loss streak: 20
Avg pips risked: 0.0016003663500678477
Remaining runs: 23
Best reward so far: -23952.752730000368

-74906.27857000194
Num buys: 1898
Num sells: 1987
Num trades: 3885
Num wins: 1029
Num losses: 2856
Win streak: 6
Loss streak: 26
Avg pips risked: 0.0010706357786357838
Remaining runs: 22
Best reward so far: -23952.752730000368

-18761.464350000282
Num buys: 461
Num sells: 469
Num trades: 930
Num wins: 389
Num losses: 541
Win streak: 7
Loss streak: 10
Avg pips risked: 0.0016098494623655937
Remaining runs: 21
Best reward so far: -18761.464350000282

-20384.543860000358
Num buys: 355
Num sells: 366
Num trades: 721
Num wins: 292
Num losses: 429
Win streak: 6
Loss streak: 10
Avg pips risked: 0.0016099445214979207
Remaining runs: 20
Best reward so far: -18761.464350000282

-16079.040560000241
Num buys: 428
Num sells: 447
Num trades: 875
Num wins: 248
Num l

In [14]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best cushion: ' + str(best_cushion))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: -16079.040560000241
Best risk/reward ratio: 2.0
Best spread: 0.1
Best cushion: 3
-----------------------
Top results:
{'reward': -18761, 'ratio': 1.0, 'spread': 0.1, 'cushion': 3}
{'reward': -20384, 'ratio': 1.0, 'spread': 0.1, 'cushion': 2}
{'reward': -16079, 'ratio': 2.0, 'spread': 0.1, 'cushion': 3}
{'reward': -23003, 'ratio': 1.5, 'spread': 0.1, 'cushion': 4}
{'reward': -21598, 'ratio': 2.0, 'spread': 0.1, 'cushion': 4}
{'reward': -19984, 'ratio': 1.5, 'spread': 0.1, 'cushion': 2}
{'reward': -23472, 'ratio': 1.5, 'spread': 0.1, 'cushion': 5}
{'reward': -22733, 'ratio': 1.0, 'spread': 0.1, 'cushion': 4}
{'reward': -18806, 'ratio': 2.0, 'spread': 0.1, 'cushion': 2}
{'reward': -17738, 'ratio': 1.5, 'spread': 0.1, 'cushion': 3}
